### Connect to drive and setup training file:

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Sep  6 11:03:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importing the libraries:

In [ ]:
installed = False
  
if not installed:
    !rm -rf indic_nlp_library indic_nlp_resources >> /dev/null
    !git clone "https://github.com/anoopkunchukuttan/indic_nlp_resources.git" --quiet
    !git clone "https://github.com/anoopkunchukuttan/indic_nlp_library" --quiet
    !pip install -r "./indic_nlp_library/requirements.txt" >> /dev/null
    !pip install indic-nlp-library >> /dev/null
    !pip install Morfessor >> /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from indicnlp.tokenize import indic_tokenize
from torch.autograd import Variable
import torch.nn.functional as F
from indicnlp import loader
from indicnlp import common
import torch.nn as nn
import numpy as np
import indicnlp
import random
import torch
import nltk
import time
import math
import copy
import sys
import csv
import gc

In [ ]:
nltk.download('punkt', quiet=True)

True

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
INDIC_NLP_LIB_HOME   =  "./indic_nlp_library"
INDIC_NLP_RESOURCES  =  "./indic_nlp_resources"

# Add indicnlp to system path:
sys.path.append(INDIC_NLP_LIB_HOME)

# Point the indicnlp resources:
common.set_resources_path(INDIC_NLP_RESOURCES)

In [ ]:
hi_word2id = {}
hi_id2word = {}
en_word2id = {}
en_id2word = {}

hi_word2freq = {}
en_word2freq = {}

### Adding Start Word and Stop Word:

In [ ]:
# Adding start, stop words in hindi vocab:
hi_word2id.update({'__<<init>>__': 0})
hi_id2word.update({0: '__<<init>>__'})
hi_word2id.update({'__<<stop>>__': 1})
hi_id2word.update({1: '__<<stop>>__'})
hi_word2id.update({'__<<unknown>>__': 2})
hi_id2word.update({2: '__<<unknown>>__'})
hi_word2id.update({'__<<padding>>__': 3})
hi_id2word.update({3: '__<<padding>>__'})

# Adding start, stop words in english vocab:
en_word2id.update({'__<<init>>__': 0})
en_id2word.update({0: '__<<init>>__'})
en_word2id.update({'__<<stop>>__': 1})
en_id2word.update({1: '__<<stop>>__'})
en_word2id.update({'__<<unknown>>__': 2})
en_id2word.update({2: '__<<unknown>>__'})
en_word2id.update({'__<<padding>>__': 3})
en_id2word.update({3: '__<<padding>>__'})

### Loading the dataset:

In [ ]:
# dataset = []
# with open('./train.csv', 'r') as file:
#     dataset = np.array([[r[1], r[2]] for r in csv.reader(file)])[1::]

In [ ]:
hi_dataset = []
with open('./drive/MyDrive/processed.hi', encoding='utf8') as file:
    hi_dataset = [s.strip('\n').replace('  ', ' ') for s in file.readlines()]

hi_dataset[0:10]

['आदि में परमेश्वर ने आकाश और पृथ्वी की सृष्टि की ।',
 'और पृथ्वी बेडौल और सुनसान पड़ी थी ; और गहरे जल के ऊपर अन्धियारा था : तथा परमेश्वर का आत्मा जल के ऊपर मण्डलाता था ।',
 'तब परमेश्वर ने कहा , उजियाला हो : तो उजियाला हो गया ।',
 'और परमेश्वर ने उजियाले को देखा कि अच्छा है ; और परमेश्वर ने उजियाले को अन्धियारे से अलग किया ।',
 'और परमेश्वर ने उजियाले को दिन और अन्धियारे को रात कहा । तथा सांझ हुई फिर भोर हुआ । इस प्रकार पहिला दिन हो गया । ।',
 'फिर परमेश्वर ने कहा , जल के बीच एक ऐसा अन्तर हो कि जल दो भाग हो जाए ।',
 'तब परमेश्वर ने एक अन्तर करके उसके नीचे के जल और उसके ऊपर के जल को अलग अलग किया ; और वैसा ही हो गया ।',
 'और परमेश्वर ने उस अन्तर को आकाश कहा । तथा सांझ हुई फिर भोर हुआ । इस प्रकार दूसरा दिन हो गया । ।',
 'फिर परमेश्वर ने कहा , आकाश के नीचे का जल एक स्थान में इकट्ठा हो जाए और सूखी भूमि दिखाई दे ; और वैसा ही हो गया ।',
 'और परमेश्वर ने सूखी भूमि को पृथ्वी कहा ; तथा जो जल इकट्ठा हुआ उसको उस ने समुद्र कहा : और परमेश्वर ने देखा कि अच्छा है ।']

In [ ]:
en_dataset = []
with open('./drive/MyDrive/processed.en', encoding='utf8') as file:
    en_dataset = [s.strip('\n').replace('  ', ' ') for s in file.readlines()]

en_dataset[0:10]

['In the beginning God created the heaven and the earth .',
 'And the earth was without form , and void ; and darkness was upon the face of the deep . And the Spirit of God moved upon the face of the waters .',
 'And God said , Let there be light : and there was light .',
 'And God saw the light , that it was good : and God divided the light from the darkness .',
 'And God called the light Day , and the darkness he called Night . And the evening and the morning were the first day .',
 'And God said , Let there be a firmament in the midst of the waters , and let it divide the waters from the waters .',
 'And God made the firmament , and divided the waters which were under the firmament from the waters which were above the firmament : and it was so .',
 'And God called the firmament Heaven . And the evening and the morning were the second day .',
 'And God said , Let the waters under the heaven be gathered together unto one place , and let the dry land appear : and it was so .',
 'And Go

In [ ]:
hi_ds = []
en_ds = []
for i in range(len(hi_dataset)):
    if len(indic_tokenize.trivial_tokenize(hi_dataset[i])) > 80:
        pass
    else:
        en_ds += [en_dataset[i]]
        hi_ds += [hi_dataset[i]]
    
hi_dataset = hi_ds
en_dataset = en_ds

In [ ]:
len(hi_dataset), len(en_dataset)

(511725, 511725)

In [ ]:
hi_word_seq = []
hi_counter = 4
for row in hi_dataset:

    # Hindi Sentences:
    temp = indic_tokenize.trivial_tokenize(row)
    hi_word_seq += [temp]
    
    for word in temp:
        if word not in hi_word2id.keys():
            hi_word2id.update({word: hi_counter})
            hi_id2word.update({hi_counter: word})
            hi_counter += 1

en_word_seq = []
en_counter = 4
for row in en_dataset:

    # English Sentences:
    temp = nltk.word_tokenize(row)
    en_word_seq += [temp]

    for word in temp:
        if word not in en_word2id.keys():
            en_word2id.update({word: en_counter})
            en_id2word.update({en_counter: word})
            en_counter += 1

In [ ]:
hi_max = max([len(l) for l in hi_word_seq])
en_max = max([len(l) for l in en_word_seq])

In [ ]:
print('Hi-Vocabulary Size:', len(hi_id2word))
print('En-Vocabulary Size:', len(en_id2word))

Hi-Vocabulary Size: 95041
En-Vocabulary Size: 90667


In [ ]:
def get_indices_seq(seq, vocab):
    
    temp = []
    for word in seq:
        if word in vocab.keys():
            temp += [vocab[word]]
        else:
            temp += [vocab['__<<unknown>>__']]

    seq = torch.tensor(
        [vocab['__<<init>>__']] + temp + [vocab['__<<stop>>__']]
    )

    return seq

In [ ]:
def get_word_seq(seq, vocab):

    temp = []
    for index in seq:
        temp += [vocab[int(index)]]

    return temp

In [ ]:
train_seq_pairs = []

for sent_id in range(len(hi_word_seq)):
    train_seq_pairs += [[
        get_indices_seq(hi_word_seq[sent_id], hi_word2id).to(device),
        get_indices_seq(en_word_seq[sent_id], en_word2id).to(device) 
    ]]

In [ ]:
train_seq_pairs_sorted = sorted(train_seq_pairs, key=lambda x: len(x[0]) + len(x[1]))

In [ ]:
BATCH_SIZE = 16

In [ ]:
input_batches = []
output_batches = []
for i in range(0, len(train_seq_pairs_sorted) - (len(train_seq_pairs_sorted) % BATCH_SIZE), BATCH_SIZE):

    hi_indices_sequences = [pair[0] for pair in train_seq_pairs_sorted[i:i+BATCH_SIZE]]
    en_indices_sequences = [pair[1] for pair in train_seq_pairs_sorted[i:i+BATCH_SIZE]]

    input_batches += [
        torch.nn.utils.rnn.pad_sequence(
            hi_indices_sequences, 
            batch_first=False, 
            padding_value=3
        )
    ]

    output_batches += [
        torch.nn.utils.rnn.pad_sequence(
            en_indices_sequences, 
            batch_first=False, 
            padding_value=3
        )
    ]

### Transformer Model:

In [ ]:
class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()
    
        self.size = d_model
        
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        
        self.eps = eps
    
    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) \
        / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm

In [ ]:
def attention(q, k, v, d_k, mask=None, dropout=None):
    
    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)
    
    if mask is not None:
        mask = mask.unsqueeze(1).to(device)
        scores = scores.masked_fill(mask == 0, -1e9)
    
    scores = F.softmax(scores, dim=-1)
    
    if dropout is not None:
        scores = dropout(scores)
        
    output = torch.matmul(scores, v)
    return output

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    def forward(self, q, k, v, mask=None):
        
        bs = q.size(0)
        
        # perform linear operation and split into N heads
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        # transpose to get dimensions bs * N * sl * d_model
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)

        # calculate attention using function we will define next
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous()\
        .view(bs, -1, self.d_model)
        output = self.out(concat)
    
        return output

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout = 0.1):
        super().__init__() 
    
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

In [ ]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.d_model = d_model
        self.embed = nn.Embedding(vocab_size, d_model)
    def forward(self, x):
        return self.embed(x)

In [ ]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len = 200, dropout = 0.1):
        super().__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(dropout)
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = \
                math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = \
                math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
 
    
    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.d_model)
        #add constant to embedding
        seq_len = x.size(1)
        pe = Variable(self.pe[:,:seq_len], requires_grad=False)
        if x.is_cuda:
            pe.cuda()
        x = x + pe
        return self.dropout(x)

In [ ]:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model, dropout=dropout)
        self.ff = FeedForward(d_model, dropout=dropout)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.norm_3 = Norm(d_model)
        
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)
        
        self.attn_1 = MultiHeadAttention(heads, d_model, dropout=dropout)
        self.attn_2 = MultiHeadAttention(heads, d_model, dropout=dropout)
        self.ff = FeedForward(d_model, dropout=dropout)

    def forward(self, x, e_outputs, src_mask, trg_mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.attn_2(x2, e_outputs, e_outputs, src_mask))
        x2 = self.norm_3(x)
        x = x + self.dropout_3(self.ff(x2))
        return x

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads, dropout):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model, dropout=dropout)
        self.layers = get_clones(EncoderLayer(d_model, heads, dropout), N)
        self.norm = Norm(d_model)
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, mask)
        return self.norm(x)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads, dropout):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model, dropout=dropout)
        self.layers = get_clones(DecoderLayer(d_model, heads, dropout), N)
        self.norm = Norm(d_model)
    def forward(self, trg, e_outputs, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, src_mask, trg_mask)
        return self.norm(x)

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab, trg_vocab, d_model, N, heads, dropout):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads, dropout)
        self.decoder = Decoder(trg_vocab, d_model, N, heads, dropout)
        self.out = nn.Linear(d_model, trg_vocab)
    def forward(self, src, trg, src_mask, trg_mask):
        e_outputs = self.encoder(src, src_mask)
        #print("DECODER")
        d_output = self.decoder(trg, e_outputs, src_mask, trg_mask)
        output = self.out(d_output)
        return output

In [ ]:
opt = {
    'SGDR': True,
    'epochs': 20,
    'd_model': 512,
    'n_layers': 6,
    'heads': 8,
    'dropout': 0.1,
    'batchsize': 32,
    'lr': 3e-4,
    'k': 5,
    'max_len': 100
}

In [ ]:
def get_model(opt, src_vocab, trg_vocab):
    
    assert opt["d_model"] % opt["heads"] == 0
    assert opt["dropout"] < 1

    model = Transformer(src_vocab, trg_vocab, opt["d_model"], opt["n_layers"], opt["heads"], opt["dropout"])
       
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    
    return model

In [ ]:
model = get_model(opt, len(hi_word2id), len(en_word2id)).to(device)
# model = get_model(opt, 95041, 90667).to('cuda')

In [ ]:
def nopeak_mask(size):
    np_mask = np.triu(np.ones((1, size, size)),
    k=1).astype('uint8')
    np_mask =  Variable(torch.from_numpy(np_mask) == 0)
    return np_mask

In [ ]:
def create_masks(src, trg):
    
    src_mask = (src != hi_word2id['__<<padding>>__']).unsqueeze(-2)

    if trg is not None:
        trg_mask = (trg != en_word2id['__<<padding>>__']).unsqueeze(-2)
        size = trg.size(1) # get seq_len for matrix
        np_mask = nopeak_mask(size).to(device)
        trg_mask = trg_mask & np_mask
    else:
        trg_mask = None
    return src_mask, trg_mask

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=opt['lr'], betas=(0.9, 0.98), eps=1e-9)

In [ ]:
model.load_state_dict(torch.load('drive/MyDrive/Dissertation models/processed_transformer_model_epoch_60'))
optimizer.load_state_dict(torch.load('drive/MyDrive/Dissertation models/processed_transformer_optim_epoch_60'))

In [ ]:
model.train()
for epoch in range(51, 61, 1):

    total_loss = 0

    indices = list(range(len(input_batches)))
    random.shuffle(indices)

    tic = time.time()

    counter = 0

    minibatch_X = input_batches[0]
    minibatch_Y = output_batches[0]

    for i in indices:

        minibatch_X = input_batches[i].T
        minibatch_Y = output_batches[i].T

        src = minibatch_X
        trg = minibatch_Y

        # print('SRC SHAPE:', src.shape)
        # print('TRG SHAPE:', trg.shape)

        trg_input = trg[:, :-1]
        # trg_input = trg[:, :]

        src_mask, trg_mask = create_masks(src, trg_input)
        preds = model(src, trg_input, src_mask, trg_mask)
        
        # print(src.shape)
        # print(trg.shape)
        # print(preds.shape)

        # ys = trg[:, :].contiguous().view(-1)
        ys = trg[:, 1:].contiguous().view(-1)
        optimizer.zero_grad()
        loss = F.cross_entropy(preds.view(-1, preds.size(-1)), ys, ignore_index=en_word2id['__<<padding>>__'])
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        counter += 1

        if counter % 200 == 0:
            print('{}/{} Batches Processed.'.format(counter, len(input_batches)))

    toc = time.time()

    # Save Weights:
    torch.save(model.state_dict(), 'drive/MyDrive/Dissertation models/processed_transformer_model_epoch_60'.format(epoch))
    torch.save(optimizer.state_dict(), 'drive/M yDrive/Dissertation models/processed_transformer_optim_epoch_60'.format(epoch))
    
    print('Epoch {}, Total Loss: {}, Total Time For This Epoch: {}'.format(epoch, total_loss, toc - tic))

In [ ]:
def init_vars(src, model):
    
    # src: indices sequence (may have padding)
    # shape: (1, 1, src_len)

    init_tok = en_word2id['__<<init>>__']
    # start token
    # shape: scalar

    src_mask = (src != hi_word2id['__<<padding>>__']).unsqueeze(-2)
    # mask padding sequences:
    # shape: (1, 1, src_len)
    # 
    # =======================================
    # Description:
    # =======================================
    # 
    # if src: [[[w1, w2, <p>, <p>, <p>]]] then src_mask: [[[True, True]]]

    e_output = model.encoder(src, src_mask)
    # encoder outputs:
    # 512 dimensional vector for each word:
    # shape: (1, src_len, 512)

    outputs = torch.LongTensor([[init_tok]]).to(device)
    # initial output for target sentence:
    # eg. [__<<init>>__]
    # shape: (1, 1)

    trg_mask = nopeak_mask(1)
    # initial output mask:
    # shape: (1, 1, 1)

    out = model.out(model.decoder(outputs, e_output, src_mask, trg_mask))
    # model output:
    # shape: (1, 1, target_vocab)

    out = F.softmax(out, dim=-1)
    # model normalized output:
    # shape: (1, 1, target_vocab)

    probs, ix = out[:, -1].data.topk(opt['k'])
    # probs: k topmost probabilities
    # shape: (1, K)

    # ix: k topmost indices
    # shape: (1, K)

    log_scores = torch.Tensor([math.log(prob) for prob in probs.data[0]]).unsqueeze(0)
    # log scores: score of the sequence so far.
    # shape: (1, K)

    outputs = torch.zeros(opt["k"], opt["max_len"]).long().to(device)
    # buffer for top k outputs: 
    # shape: (K, max_len)

    outputs[:, 0] = init_tok
    outputs[:, 1] = ix[0]
    # set first word = '__<<init>>__' and predict next word:
    # shape: (K, max_len)
    
    e_outputs = torch.zeros(opt["k"], e_output.size(-2), e_output.size(-1)).to(device)
    # allocate tensor for encoder outputs
    # shape: (K, src_len, 512)

    e_outputs[:, :] = e_output[0]
    # broadcast encoder outputs for each sequences
    # shape(K, src_len, 512)

    return outputs, e_outputs, log_scores

In [ ]:
def k_best_outputs(outputs, out, log_scores, i, k):
    
    # outputs: (K, max_len)
    # out: (K, i, target_vocab)
    # log_scores: (1, K)
    # i: current length so far.
    # k: beam width.

    probs, ix = out[:, -1].data.topk(k)
    # probs: top-k predictions for each of top k previous probabilities
    # shape: (k, k)

    # ix: top-k indices for each of top k previous probabilities
    # shape: (k, k)

    log_probs = torch.Tensor([math.log(p) for p in probs.data.view(-1)]).view(k, -1) + log_scores.transpose(0, 1)
    # calculating log_probabilities:
    # shape: (k, k)
    
    k_probs, k_ix = log_probs.view(-1).topk(k)
    # k_probs shape: (5,)
    # k_ix shape: (5,)

    row = k_ix // k
    col = k_ix % k

    outputs[:, :i] = outputs[row, :i]
    outputs[:, i] = ix[row, col]

    log_scores = k_probs.unsqueeze(0)

    # outputs: (K, max_len)
    # log_scores: (1, K)
    
    return outputs, log_scores

In [ ]:
def beam_search(src, model, opt):

    outputs, e_outputs, log_scores = init_vars(src, model)
    # initialize the generation of sentence and allocate buffers
    # outputs: (K, max_len)
    # e_outputs: (K, seq_len, 512)
    # log_scores: (1, K)

    eos_tok = en_word2id['__<<stop>>__']
    # target '__<<stop>>__' token:
    # shape: scalar

    src_mask = (src != hi_word2id['__<<padding>>__']).unsqueeze(-2)
    # mask padding sequences:
    # shape: (1, 1, src_len)
    # 
    # =======================================
    # Description:
    # =======================================
    # 
    # if src: [[[w1, w2, <p>, <p>, <p>]]] then src_mask: [[[True, True]]]

    ind = None
    for i in range(2, opt["max_len"]):
    
        trg_mask = nopeak_mask(i)
        # Masking the target upto current length:
        # shape: (1, i, i) where i represents output length so far.

        out = model.out(model.decoder(outputs[:,:i], e_outputs, src_mask, trg_mask))
        # out: (K, i, dict) where i represents output length so far.

        out = F.softmax(out, dim=-1)
        # softmax: (K, i, dict) where i represents output length so far.

        outputs, log_scores = k_best_outputs(outputs, out, log_scores, i, opt["k"])
        # finding next K best words:
        # outputs: (K, max_len)
        # log_scores: (1, K)

        ones = (outputs.long()==eos_tok).nonzero()

        sentence_lengths = torch.zeros(len(outputs), dtype=torch.long).cuda()
        for vec in ones:
            i = vec[0]
            if int(sentence_lengths[i]) == 0:
                sentence_lengths[i] = vec[1]

        num_finished_sentences = len([s for s in sentence_lengths if s > 0])

        if num_finished_sentences == opt["k"]:
            alpha = 0.7
            div = 1/(sentence_lengths.type_as(log_scores)**alpha)
            _, ind = torch.max(log_scores * div, 1)
            ind = ind.data[0]
            break

    # print(num_finished_sentences)

    if ind is None:
        length = (outputs.long()[0]==eos_tok).nonzero()[0]
        return ' '.join([en_id2word[int(tok)] for tok in outputs[0][1:length]])
    
    else:
        length = (outputs.long()[ind]==eos_tok).nonzero()[0]
        return ' '.join([en_id2word[int(tok)] for tok in outputs[ind][1:length]])

In [ ]:
def translate_sentence(sentence, model, opt):
    
    model.eval()
    
    indexed = []
    sentence = indic_tokenize.trivial_tokenize(sentence)
    
    for tok in sentence:
        if tok in hi_word2id.keys():
            if hi_word2id[tok] != 0:
                indexed.append(hi_word2id[tok])
        else:
            indexed.append(hi_word2id['__<<unknown>>__'])

    sentence = torch.Tensor([[0] + indexed + [1]]).long().to(device)

    # print('SENT', sentence)

    sentence = beam_search(sentence, model, opt)

    return sentence

In [ ]:
target = input('Enter Hindi Input: ')
print(f'\nTranslated English sentence : {translate_sentence(target, model, opt)}')

Enter Hindi Input: मैं आज यहां भारत की जनता की संवेदना और प्रार्थना के साथ आया हूं ।

Translated English sentence : I am here with the condolences and prayers of the people of India .


In [ ]:
target = input('Enter Hindi Input: ')
print(f'\nTranslated English sentence : {translate_sentence(target, model, opt)}')

Enter Hindi Input: वह एक ऐसा जानवर है जो पैसे के लिए किसी भी हद तक गिर सकता है

Translated English sentence : He is a wild beast that may fall to a certain degree of money


In [ ]:
target = input('Enter Hindi Input: ')
print(f'\nTranslated English sentence : {translate_sentence(target, model, opt)}')

Enter Hindi Input: मैं उच्च अध्ययन के लिए कनाडा जा रहा हूँ।

Translated English sentence : I am going to Canada for a higher study .


In [ ]:
target = input('Enter Hindi Input: ')
print(f'\nTranslated English sentence : {translate_sentence(target, model, opt)}')

Enter Hindi Input: मुझे कल दिल्ली जाना पड़ सकता है।

Translated English sentence : I can get to Delhi tomorrow .


In [ ]:
target = input('Enter Hindi Input: ')
print(f'\nTranslated English sentence : {translate_sentence(target, model, opt)}')

Enter Hindi Input: मैंने अपना स्नातक अच्छे परिणाम के साथ पूरा किया है

Translated English sentence : I have completed my graduating with good results


In [ ]:
target = "मैं आज यहां भारत की जनता की संवेदना और प्रार्थना के साथ आया हूं ।"
print('Hindi Input: ', target)
print('\nEnglish Translation: ', translate_sentence(target, model, opt))

Hindi Input:  मैं आज यहां भारत की जनता की संवेदना और प्रार्थना के साथ आया हूं ।

English Translation:  I am here with the condolences and prayers of the people of India .


In [ ]:
import io
data_path = "/content/drive/MyDrive/test_dataset.txt"
trgs = []
src = []
pred_trgs = []

# Read in the data.
lines = io.open(data_path, encoding = "utf-8").read().split("\n")
lines  = lines[:-1]
lines = [line.split("\t") for line in lines]
src = [line[1] for line in lines]
trgs = [line[0] for line in lines]

for sentence in src:
  pred_trg = translate_sentence(sentence, model, opt)
  pred_trgs.append(pred_trg)

pred_trgs = [pred_trg.split() for pred_trg in pred_trgs]
trgs = [trg.split() for trg in trgs]

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
score = corpus_bleu(trgs, pred_trgs)
print(score)

0.6208142387916832


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
